# PrepareData 1 - TS_S, TsSs_S, SS_S, SsSs_S

In [1]:
import pandas as pd

from sklearn.model_selection import KFold
from sklearn import preprocessing

import numpy as np

import os

In [2]:
for method in ['TS_S_', 'TsSs_S_', 'SS_S_', 'SsSs_S_']:
    if not os.path.exists(f'./{method}PreparedData'):
        os.mkdir(f'./{method}PreparedData')

In [3]:
kf = KFold(n_splits = 5, shuffle = True, random_state=42)

In [11]:
def manipdata(newStdData, NWEEKS, meth, method, score, discard):
    
    metadata = ['ID', 'Domain', 'Week', 'Selfweeks']
    orig = ['Assets', 'Actions', 'Competency',
           'Form_record', 'Form_template', 'Incident', 'Users']
    

    skip = 0
    if discard:
        skip = 26
       
    
    if method == 'SsSs_S':
        masterlist = [list() for i in range(7*NWEEKS+6)]
        
        for i in range(len(newStdData)):
            if (newStdData.loc[i]['Selfweeks'] > NWEEKS+skip) and (i+1 < len(newStdData)) and (newStdData.loc[i]['Domain'] == newStdData.loc[i+1]['Domain']):

                for j in range(len(metadata)):
                    masterlist[j].append(newStdData.loc[i][metadata[j]])

                for j in range(len(orig)):
                    for k in range(NWEEKS):
                        masterlist[j*NWEEKS+4+k].append(newStdData.loc[i-k][orig[j]])
                
                masterlist[7*NWEEKS+4].append(newStdData.loc[i][f'Score{score}'])
                masterlist[7*NWEEKS+5].append(newStdData.loc[i+1][f'Score{score}'])

        out = pd.DataFrame()

        for i in range(7*NWEEKS+6):
            if i < 4:
                out.insert(i, metadata[i], masterlist[i])
            elif i == 7*NWEEKS + 4:
                out.insert(i, 'PrevTarget', masterlist[i])
            elif i == 7*NWEEKS + 5:
                out.insert(i, 'Target', masterlist[i])
            else:
                print(f'{orig[int((i-4)/NWEEKS)]} {(i-4)%NWEEKS}-')
                out.insert(i, f'{orig[int((i-4)/NWEEKS)]} {(i-4)%NWEEKS}-', masterlist[i])

        out.to_csv(f'./{method}_PreparedData/{method}_{meth}_{score}_{NWEEKS}.csv', index = False)
    
    
    elif method == 'TsSs_S':
        masterlist = [list() for i in range(7*NWEEKS+6)]
        
        for i in range(len(newStdData)):
            if (newStdData.loc[i]['Selfweeks'] > NWEEKS+skip) and (i+1 < len(newStdData)) and (newStdData.loc[i]['Domain'] == newStdData.loc[i+1]['Domain']):

                for j in range(len(metadata)):
                    masterlist[j].append(newStdData.loc[i][metadata[j]])

                for j in range(len(orig)):
                    for k in range(NWEEKS):
                        if not k:
                            masterlist[j*NWEEKS+4+k].append(newStdData.loc[i][orig[j]])
                        else:
                            masterlist[j*NWEEKS+4+k].append(newStdData.loc[i-k][f'trend{score}'])
                
                masterlist[7*NWEEKS+4].append(newStdData.loc[i][f'Score{score}'])
                masterlist[7*NWEEKS+5].append(newStdData.loc[i+1][f'Score{score}'])

        out = pd.DataFrame()

        for i in range(7*NWEEKS+6):
            if i < 4:
                out.insert(i, metadata[i], masterlist[i])
            elif i == 7*NWEEKS + 5:
                out.insert(i, 'Target', masterlist[i])
            elif i == 7*NWEEKS + 4:
                out.insert(i, 'PrevTarget', masterlist[i])
            else:
                out.insert(i, f'{orig[int((i-4)/NWEEKS)]} {(i-4)%NWEEKS}-', masterlist[i])

        out.to_csv(f'./{method}_PreparedData/{method}_{meth}_{score}_{NWEEKS}.csv', index = False)
    
    
    elif method == 'SS_S':
        
        masterlist = [list() for i in range(NWEEKS+6)]

        for i in range(len(newStdData)):
            if (newStdData.loc[i]['Selfweeks'] > NWEEKS+skip) and (i+1 < len(newStdData)) and (newStdData.loc[i]['Domain'] == newStdData.loc[i+1]['Domain']):

                for j in range(len(metadata)):
                    masterlist[j].append(newStdData.loc[i][metadata[j]])

                for j in range(4, NWEEKS+4):
                    masterlist[j].append(newStdData.loc[i-(j-4)][f'Score{score}'])
                
                masterlist[NWEEKS+4].append(newStdData.loc[i][f'Score{score}'])
                masterlist[NWEEKS+5].append(newStdData.loc[i+1][f'Score{score}'])

        out = pd.DataFrame()

        for i in range(NWEEKS+6):
            if i < 4:
                out.insert(i, metadata[i], masterlist[i])
            elif i == NWEEKS + 4:
                out.insert(i, 'PrevTarget', masterlist[i])
            elif i == NWEEKS + 5:
                out.insert(i, 'Target', masterlist[i])
            else:
                out.insert(i, f'{i-4}', masterlist[i])

        out.to_csv(f'./{method}_PreparedData/{method}_{meth}_{score}_{NWEEKS}.csv', index = False)
    
    
    elif method == 'TS_S':
        
        masterlist = [list() for i in range(NWEEKS+6)]

        for i in range(len(newStdData)):
            if (newStdData.loc[i]['Selfweeks'] > NWEEKS+skip) and (i+1 < len(newStdData)) and (newStdData.loc[i]['Domain'] == newStdData.loc[i+1]['Domain']):

                for j in range(len(metadata)):
                    masterlist[j].append(newStdData.loc[i][metadata[j]])

                for j in range(5, NWEEKS+4):
                    masterlist[j].append(newStdData.loc[i-(j-4)][f'trend{score}'])
                
                masterlist[4].append(newStdData.loc[i][f'Score{score}'])
                masterlist[NWEEKS+4].append(newStdData.loc[i][f'Score{score}'])
                masterlist[NWEEKS+5].append(newStdData.loc[i+1][f'Score{score}'])

        out = pd.DataFrame()

        for i in range(NWEEKS+6):
            if i < 4:
                out.insert(i, metadata[i], masterlist[i])
            elif i == NWEEKS + 4:
                out.insert(i, 'PrevTarget', masterlist[i])
            elif i == NWEEKS + 5:
                out.insert(i, 'Target', masterlist[i])
            else:
                out.insert(i, f'{i-4}', masterlist[i])

        out.to_csv(f'./{method}_PreparedData/{method}_{meth}_{score}_{NWEEKS}.csv', index = False)


In [12]:
for meth in ['A', '-26']:
    df = pd.read_csv('../Initial/Partial_Output/_3_combined_cleaned_data.csv')
    df = df[df.columns[:-1]]
    
    discard = False
    
    if '-26' in meth:
        df = df[df['Selfweeks'] > 26]
        df.index = range(len(df))
        
        discard = True
    
    data = pd.DataFrame()
    for domain, tmp in df.groupby('Domain'):
        tmp.index = range(len(tmp))
        
        for i in range(len(tmp)-1, -1, -1):
            if ((tmp.loc[i]['Assets']) | (tmp.loc[i]['Actions']) | (tmp.loc[i]['Competency']) |
                    (tmp.loc[i]['Form_record']) | (tmp.loc[i]['Form_template']) |
                    (tmp.loc[i]['Incident']) | (tmp.loc[i]['Users'])):
                break
        
        data = pd.concat([data, tmp[0:i+1]])
        
    stdData = pd.DataFrame()
    
    for domain, compData in data.groupby('Domain'):
    
        compDataX = compData[['Assets', 'Actions', 'Competency',
           'Form_record', 'Form_template', 'Incident', 'Users']]

        scaler = preprocessing.StandardScaler().fit(compDataX)

        compData[['Assets', 'Actions', 'Competency',
           'Form_record', 'Form_template', 'Incident', 'Users']] = scaler.transform(compDataX)

        stdData = pd.concat([stdData, compData])
    
    stdData['Score2_1'] = stdData['Users'] + stdData['Actions']
    stdData['Score3_1'] = stdData['Score2_1'] + stdData['Form_record']
    
    a = stdData[['Users', 'Score2_1']].corr().loc['Users']['Score2_1']
    b = stdData[['Actions', 'Score2_1']].corr().loc['Actions']['Score2_1']
    c = stdData[['Users', 'Score3_1']].corr().loc['Users']['Score3_1']
    d = stdData[['Actions', 'Score3_1']].corr().loc['Actions']['Score3_1']
    e = stdData[['Form_record', 'Score3_1']].corr().loc['Form_record']['Score3_1']
    
    stdData['Score2_2'] = a * stdData['Users'] + b * stdData['Actions']
    stdData['Score3_2'] = c * stdData['Users'] + d * stdData['Actions'] + e * stdData['Form_record']
    
    trend2_1 = list()
    trend3_1 = list()
    trend2_2 = list()
    trend3_2 = list()
    
    for domain, compData in stdData.groupby('Domain'):

        compData.index = compData['Selfweeks']

        index = compData.index

        out1 = [(compData.loc[i+1]['Score2_1']-compData.loc[i]['Score2_1']) if (i+1 in index)
               else np.nan for i in index]
        out2 = [(compData.loc[i+1]['Score3_1']-compData.loc[i]['Score3_1']) if (i+1 in index)
               else np.nan for i in index]
        out3 = [(compData.loc[i+1]['Score2_2']-compData.loc[i]['Score2_2']) if (i+1 in index)
               else np.nan for i in index]
        out4 = [(compData.loc[i+1]['Score3_2']-compData.loc[i]['Score3_2']) if (i+1 in index)
               else np.nan for i in index]
        
        trend2_1.extend(out1)
        trend3_1.extend(out2)
        trend2_2.extend(out3)
        trend3_2.extend(out4)

    stdData[f'trend2_1'] = trend2_1
    stdData[f'trend3_1'] = trend3_1
    stdData[f'trend2_2'] = trend2_2
    stdData[f'trend3_2'] = trend3_2
    
    stdData['Week'].astype(int)
    stdData = stdData.sort_values(['Domain', 'Week'])
    stdData.index = range(len(stdData))
    print(stdData)
    for method in ['TS_S', 'TsSs_S', 'Ss_S', 'SsSs_S']:
        for score in ['2_1', '3_1', '2_2', '3_2']:
            for nWeeks in range(2, 13):
                manipdata(stdData, nWeeks, meth, method, score, discard)
        

                ID    Domain  Week  Selfweeks    Assets   Actions  Competency  \
0      abdgroup 24  abdgroup    24          1 -0.086066 -0.573627   -0.091484   
1      abdgroup 25  abdgroup    25          2 -0.086066 -0.573627   -0.091484   
2      abdgroup 26  abdgroup    26          3 -0.086066 -0.573627   -0.091484   
3      abdgroup 27  abdgroup    27          4 -0.086066 -0.573627   -0.091484   
4      abdgroup 28  abdgroup    28          5 -0.086066 -0.573627   -0.091484   
...            ...       ...   ...        ...       ...       ...         ...   
20528   yurika 211    yurika   211         59  5.342019  2.366796    3.067270   
20529   yurika 212    yurika   212         60  2.772202 -0.249136    0.529632   
20530   yurika 213    yurika   213         61  4.353628 -0.249136   -0.309257   
20531   yurika 214    yurika   214         62  0.993098 -0.249136    2.165464   
20532   yurika 215    yurika   215         63  1.388454  1.494819    3.109214   

       Form_record  Form_te

In [6]:
for method in ['TS_S_', 'TsSs_S_', 'SS_S_', 'SsSs_S_']:
    if not os.path.exists(f'./{method}SplitData'):
        os.mkdir(f'./{method}SplitData')

In [7]:
for method in ['TS_S_', 'TsSs_S_', 'SS_S_', 'SsSs_S_']:    
    filelist = os.listdir(f"./{method}PreparedData")
    for file in filelist:
        if file[0] != '.':
            stdData = pd.read_csv(f"./{method}PreparedData/{file}")
            fold = 1
            
            startweek = int(file.strip('.csv').split('_')[-1])

            if '-26' in file:
                startweek += 26

            for trainWeeks, testWeeks in kf.split(range(startweek, 215)):

                testBool = stdData.Week.isin(testWeeks)
                testData = stdData[testBool]

                trainBool = stdData.Week.isin(trainWeeks)
                trainData = stdData[trainBool]

                testData.to_csv(f'./{method}SplitData/{file.strip(".csv")}_Test_{fold}.csv', index = False)
                trainData.to_csv(f'./{method}SplitData/{file.strip(".csv")}_Train_{fold}.csv', index = False)

                fold += 1
    

FileNotFoundError: [Errno 2] No such file or directory: './TS_S_SplitData/TS_S_A_3_2_1_Test_1.csv'